# Connexion

In [2]:
import sqlite3
import pandas as pd 
from datetime import datetime
import dateutil.relativedelta

conn = sqlite3.connect('data/olist.db')

cursor = conn.cursor()

In [3]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Tables dans la base de données :", tables)

Tables dans la base de données : [('customers',), ('geoloc',), ('order_items',), ('order_pymts',), ('order_reviews',), ('orders',), ('products',), ('sellers',), ('translation',)]


# First request

In [4]:
request = f"""SELECT * FROM orders 
                WHERE order_status != 'canceled' 
                AND order_purchase_timestamp > DATE((SELECT MAX(order_purchase_timestamp) FROM orders), '-3 months')
                 AND julianday(order_delivered_customer_date) - julianday(order_estimated_delivery_date) >= 3"""
df = pd.read_sql_query(request, conn)

In [5]:
df

,index,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,561,cfa4fa27b417971e86d8127cb688712f,7093250e1741ebbed41f0cc552025fd6,delivered,2018-08-16 09:44:23,2018-08-17 03:10:22,2018-08-17 18:55:00,2018-08-29 01:41:41,2018-08-22 00:00:00
1,711,234c056c50619f48da64f731c48242b4,44e460a655f7154ccd9faa4dbbbaf68a,delivered,2018-08-14 14:49:15,2018-08-15 03:15:15,2018-08-31 15:25:00,2018-09-01 18:14:42,2018-08-23 00:00:00
2,728,8ad3f1d0f96992e43566c4c82c9f6c58,948b29e24216a05fea13a18d8db45ea5,delivered,2018-07-17 21:25:29,2018-07-17 21:35:17,2018-07-18 13:08:00,2018-08-14 04:04:40,2018-08-03 00:00:00
3,1844,7f579e203c931f3e8410103359c6d523,d665be250d1c687c58fdea61a9b55a58,delivered,2018-08-02 18:09:27,2018-08-02 18:43:43,2018-08-10 14:15:00,2018-08-13 20:11:47,2018-08-09 00:00:00
4,1967,cb6e441ff2ef574ce08d3709426f88ec,4fb843d304c57182d4aa27bb39ca592b,delivered,2018-08-08 19:27:03,2018-08-08 19:44:59,2018-08-10 13:32:00,2018-08-18 01:11:58,2018-08-15 00:00:00
...,...,...,...,...,...,...,...,...,...
317,98368,874a7690bc049bd4ce210d195bdfff7b,f6b2c83ea477af36f44a1e2df7320036,delivered,2018-08-02 12:06:47,2018-08-02 13:05:56,2018-08-10 14:42:00,2018-08-13 15:50:48,2018-08-08 00:00:00
318,98382,587e32dd528769d669d539531d32aeb5,0af8a25fb0b6f833724bb5818275807b,delivered,2018-08-10 11:46:09,2018-08-11 02:50:25,2018-08-14 10:09:00,2018-09-03 09:32:31,2018-08-28 00:00:00
319,98564,3adb141ba4bd69dd7fe8d3fb733c6b74,c0539d5c87fc7c97a8418adffe4b45f0,delivered,2018-08-14 23:29:21,2018-08-16 03:05:11,2018-08-16 13:28:00,2018-08-28 18:02:52,2018-08-24 00:00:00
320,99279,4cf09d9e5ebbe0f91ddd7bf9aae891cd,07b6b4fe5fefb948fc76b6d2bdba77d8,delivered,2018-07-19 08:37:26,2018-07-21 03:25:17,2018-07-23 15:31:00,2018-08-21 01:12:45,2018-08-10 00:00:00


# Second request

In [6]:
('customers',), ('geoloc',), ('order_items',), ('order_pymts',), ('order_reviews',), ('orders',), ('products',), ('sellers',), ('translation',)

(('customers',),
 ('geoloc',),
 ('order_items',),
 ('order_pymts',),
 ('order_reviews',),
 ('orders',),
 ('products',),
 ('sellers',),
 ('translation',))

In [7]:
request = f"SELECT seller_id, sum(price) as CA FROM order_items GROUP BY seller_id HAVING CA > 100000"
df = pd.read_sql_query(request, conn)
df

,seller_id,CA
0,1025f0e2d44d7041d6cf58b6550e0bfa,138968.55
1,1f50f920176fa81dab994f9023523100,106939.21
2,46dc3b2cc0980fb8ec44634e21d2718e,128111.19
3,4869f7a5dfa277a7dca6462dcf3b52b2,229472.63
4,4a3ca9315b744ce9f8e9374361493884,200472.92
5,53243585a1d6dc2643021fd1853d8905,222776.05
6,5dceca129747e92ff8ef7a997dc4f8ca,112155.53
7,620c87c171fb2a6dd6e8bb4dec959fc6,114774.50
8,6560211a19b47992c3666cc44a7e94c0,123304.83
9,7a67c85e85bb2ce8582c35f2203ad736,141745.53


## third request

Tables dans la base de données : [('customers',), ('geoloc',), ('order_items',), ('order_pymts',), ('order_reviews',), ('orders',), ('products',), ('sellers',), ('translation',)]

In [8]:
request = f"""SELECT DISTINCT order_id, seller_id FROM order_items ;"""
df = pd.read_sql_query(request, conn)
df

,order_id,seller_id
0,00010242fe8c5a6d1ba2dd792cb16214,48436dade18ac8b2bce089ec2a041202
1,00018f77f2f0320c557190d7a144bdd3,dd7ddc04e1b6c2c614352b383efe2d36
2,000229ec398224ef6ca0657da4fc703e,5b51032eddd242adc84c38acab88f23d
3,00024acbcdf0a6daa1e931b038114c75,9d7a1d34a5052409006425275ba1c2b4
4,00042b26cf59d7ce69dfabb4e55b4fd9,df560393f3a51e74553ab94004ba5c87
...,...,...
100005,fffc94f6ce00a00581880bf54a75a037,b8bc237ba3788b23da09c0f1f3a3288c
100006,fffcd46ef2263f404302a634eb57f7eb,f3c38ab652836d21de61fb8314b69182
100007,fffce4705a9662cd70adb13d4a31832d,c3cfdc648177fdbbbb35635a37472c53
100008,fffe18544ffabc95dfada21779c9644f,2b3e4a2a3ea8e01938cabda2a3e5cc79


In [9]:
request = f"""SELECT * FROM order_items;"""
df = pd.read_sql_query(request, conn)
df

,index,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14
...,...,...,...,...,...,...,...,...
112645,112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41
112646,112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53
112647,112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95
112648,112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72


In [10]:
request = f"""SELECT * FROM orders AS A
              LEFT JOIN (
                  SELECT order_id, seller_id 
                  FROM order_items
                  ) AS B ON A.order_id = B.order_id;"""
df = pd.read_sql_query(request, conn)
df

,index,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_id,seller_id
0,0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,e481f51cbdc54678b7cc49136f2d6af7,3504c0cb71d7fa48d967e0e4c94d59d9
1,1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,53cdb2fc8bc7dce0b6741e2150273451,289cdb325fb7e7f891c38608bf9e0962
2,2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,47770eb9100c2d0c44946d9cf07ec65d,4869f7a5dfa277a7dca6462dcf3b52b2
3,3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,949d5b44dbf5de918fe9c16f97b45f8a,66922902710d126a0e7d26b0e3805106
4,4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,ad21c59c0840e6cb83a9ceb5573f8159,2c9e548be18521d1c43cde1c582c6de8
...,...,...,...,...,...,...,...,...,...,...,...
113420,99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02 00:00:00,63943bddc261676b46f01ca7ac2f7bd8,1f9ab4708f3056ede07124aad39a2554
113421,99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27 00:00:00,83c1379a015df1e13d02aae0204711ab,d50d79cb34e38265a8649c383dcffd48
113422,99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15 00:00:00,11c177c8e97725db2631073c19f07b62,a1043bafd471dff536d0c462352beb48
113423,99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15 00:00:00,11c177c8e97725db2631073c19f07b62,a1043bafd471dff536d0c462352beb48


In [11]:
request = f"SELECT * FROM sellers;"
df = pd.read_sql_query(request, conn)
df

,index,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP
...,...,...,...,...,...
3090,3090,98dddbc4601dd4443ca174359b237166,87111,sarandi,PR
3091,3091,f8201cab383e484733266d1906e2fdfa,88137,palhoca,SC
3092,3092,74871d19219c7d518d0090283e03c137,4650,sao paulo,SP
3093,3093,e603cf3fec55f8697c9059638d6c8eb5,96080,pelotas,RS


In [12]:
request = f"""SELECT *
FROM order_items ORDER BY shipping_limit_date DESC;"""
df = pd.read_sql_query(request, conn)
df

,index,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,85729,c2bb89b5c1dd978d507284be78a04cb2,1,87b92e06b320e803d334ac23966c80b1,7a241947449cc45dbfda4f9d0798d9d0,2020-04-09 22:35:08,99.99,61.44
1,85730,c2bb89b5c1dd978d507284be78a04cb2,2,87b92e06b320e803d334ac23966c80b1,7a241947449cc45dbfda4f9d0798d9d0,2020-04-09 22:35:08,99.99,61.44
2,8643,13bdf405f961a6deec817d817f5c6624,1,96ea060e41bdecc64e2de00b97068975,7a241947449cc45dbfda4f9d0798d9d0,2020-02-05 03:30:51,69.99,14.66
3,68516,9c94a4ea2f7876660fa6f1b59b69c8e6,1,282b126b2354516c5f400154398f616d,7a241947449cc45dbfda4f9d0798d9d0,2020-02-03 20:23:22,75.99,14.70
4,26104,3b61aab5de69abc1731138bd104a777f,1,6aa063e063f2ab982b471e58afe06d72,610f72e407cdd7caaa2f8167b0163fd8,2018-09-18 21:10:15,999.99,24.77
...,...,...,...,...,...,...,...,...
112645,90368,cd3b8574c82b42fc8129f6d502690c3e,1,e2a1d45a73dc7f5a7f9236b043431b89,b499c00f28f4b7069ff6550af8c1348a,2016-10-08 10:34:01,29.99,10.96
112646,84389,bfbd0f9bdef84302105ad712db648a6c,1,5a6b04657a4c5ee34285d1e4619a96b4,ecccfa2bb93b34a3bf033cc5d1dcdc69,2016-09-19 23:11:33,44.99,2.83
112647,84390,bfbd0f9bdef84302105ad712db648a6c,2,5a6b04657a4c5ee34285d1e4619a96b4,ecccfa2bb93b34a3bf033cc5d1dcdc69,2016-09-19 23:11:33,44.99,2.83
112648,84391,bfbd0f9bdef84302105ad712db648a6c,3,5a6b04657a4c5ee34285d1e4619a96b4,ecccfa2bb93b34a3bf033cc5d1dcdc69,2016-09-19 23:11:33,44.99,2.83


In [13]:
request = f"""SELECT seller_id
                FROM order_items
                GROUP BY seller_id
                HAVING MIN(DATE(shipping_limit_date)) >= '2020-01-09';"""
df = pd.read_sql_query(request, conn)
df

,seller_id


In [14]:
request = f"""SELECT *
                FROM customers;"""
df = pd.read_sql_query(request, conn)
df

,index,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP
...,...,...,...,...,...,...
99436,99436,17ddf5dd5d51696bb3d7c6291687be6f,1a29b476fee25c95fbafc67c5ac95cf8,3937,sao paulo,SP
99437,99437,e7b71a9017aa05c9a7fd292d714858e8,d52a67c98be1cf6a5c84435bd38d095d,6764,taboao da serra,SP
99438,99438,5e28dfe12db7fb50a4b2f691faecea5e,e9f50caf99f032f0bf3c55141f019d99,60115,fortaleza,CE
99439,99439,56b18e2166679b8a959d72dd06da27f9,73c2643a0a458b49f58cea58833b192e,92120,canoas,RS


In [15]:
request = f"""SELECT *
                FROM order_reviews;"""
df = pd.read_sql_query(request, conn)
df

,index,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,None,None,2018-01-18 00:00:00,2018-01-18 21:46:59
1,1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,None,None,2018-03-10 00:00:00,2018-03-11 03:05:13
2,2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,None,None,2018-02-17 00:00:00,2018-02-18 14:36:24
3,3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,None,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,None,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53
...,...,...,...,...,...,...,...,...
99219,99219,574ed12dd733e5fa530cfd4bbf39d7c9,2a8c23fee101d4d5662fa670396eb8da,5,None,None,2018-07-07 00:00:00,2018-07-14 17:18:30
99220,99220,f3897127253a9592a73be9bdfdf4ed7a,22ec9f0669f784db00fa86d035cf8602,5,None,None,2017-12-09 00:00:00,2017-12-11 20:06:42
99221,99221,b3de70c89b1510c4cd3d0649fd302472,55d4004744368f5571d1f590031933e4,5,None,"Excelente mochila, entrega super rápida. Super...",2018-03-22 00:00:00,2018-03-23 09:10:43
99222,99222,1adeb9d84d72fe4e337617733eb85149,7725825d039fc1f0ceb7635e3f7d9206,4,None,None,2018-07-01 00:00:00,2018-07-02 12:59:13


In [16]:
request = f"""SELECT *
                FROM orders;"""
df = pd.read_sql_query(request, conn)
df

,index,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00
...,...,...,...,...,...,...,...,...,...
99436,99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28 00:00:00
99437,99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02 00:00:00
99438,99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27 00:00:00
99439,99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15 00:00:00


## 4th request

In [17]:
request = f"""SELECT AVG(A.review_score) as AVG_review, C.customer_zip_code_prefix, count(C.customer_zip_code_prefix) as count_zip
                FROM (
                    SELECT review_score, review_id, order_id, review_creation_date FROM order_reviews
                    ) as A
                LEFT JOIN (
                    SELECT order_id, customer_id, order_purchase_timestamp  FROM orders
                    WHERE order_purchase_timestamp >= DATE(
                    (SELECT MAX(order_purchase_timestamp) FROM orders), 
                    '-12 months')
                    ) as B on A.order_id = B.order_id
                LEFT JOIN(
                    SELECT customer_id, customer_zip_code_prefix FROM customers
                ) as C on B.customer_id = C.customer_id
                GROUP BY C.customer_zip_code_prefix
                HAVING count_zip > 30
                ORDER BY AVG_review
                LIMIT 5;"""
df = pd.read_sql_query(request, conn)
df

,AVG_review,customer_zip_code_prefix,count_zip
0,2.808511,22753,47
1,3.135135,22770,37
2,3.233333,22793,90
3,3.277778,21321,36
4,3.351351,22780,37


In [18]:
request = f"""SELECT AVG(A.review_score) as AVG_review, C.customer_zip_code_prefix, count(C.customer_zip_code_prefix) as count_zip
                FROM (
                    SELECT review_score, review_id, order_id, review_creation_date FROM order_reviews
                    WHERE review_creation_date >= DATE(
                    (SELECT MAX(review_creation_date) FROM order_reviews), 
                    '-12 months')
                    ) as A
                LEFT JOIN (
                    SELECT order_id, customer_id FROM orders
                    ) as B on A.order_id = B.order_id
                LEFT JOIN(
                    SELECT customer_id, customer_zip_code_prefix FROM customers
                ) as C on B.customer_id = C.customer_id
                GROUP BY C.customer_zip_code_prefix
                HAVING count_zip > 30
                ORDER BY AVG_review
                LIMIT 5;"""
df = pd.read_sql_query(request, conn)
df

,AVG_review,customer_zip_code_prefix,count_zip
0,2.867925,22753,53
1,3.000000,22723,31
2,3.125000,28893,32
3,3.184211,22770,38
4,3.272727,13056,33


## tables to csv

In [19]:
for table in tables:
    request = f"""SELECT * FROM {table[0]};"""
    df = pd.read_sql_query(request, conn)
    df.drop(columns=["index"], inplace=True)
    df.to_csv(f"data/{table[0]}.csv", index=False)